In [ ]:
#all the imports
import torch
import torch.nn as nn


In [ ]:
#load and originze data




In [ ]:
#make the Model 

#initialize weights 
def _init_weights(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')  # He init
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

#overall model
class StockMovementMLP(nn.Module):
    def __init__(self, T: int):
        super().__init__()
        input_dim = 20 * T
        hidden_dims = [256, 128, 64, 32]  # try a different one [256,512,64,32]
        layers = []

        dims = [input_dim] + hidden_dims
        for i in range(len(dims)-1):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            layers.append(nn.ReLU())
            #--------------------------#
            layers.append(nn.Dropout(0.2))  # Regularization make another model without this and see how it does
            #==========================#
        self.model = nn.Sequential(*layers)
        self.output_layer = nn.Linear(hidden_dims[-1], 7)  # one output per time horizon

        self._init_weights() # initialize weights here

    def forward(self, x):
        x = self.model(x)
        return self.output_layer(x)  # [batch_size, 7]


In [ ]:
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

def train_model(model, train_loader, val_loader, num_epochs=50, lr=1e-3, device='cuda'):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                val_pred = model(X_val)
                val_loss += criterion(val_pred, y_val).item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        # Save best model
        if epoch % 5 == 0:
            torch.save(model.state_dict(), f'checkpoint_epoch{epoch}.pth')

        # Print stats
        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Plot losses
        plt.figure(figsize=(8, 4))
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training vs Validation Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Ask user if they want to stop
        stop = input("Type 'stop' to end training early, or press Enter to continue: ")
        if stop.strip().lower() == 'stop':
            print("Stopping early by user request.")
            break

    print("Training finished.")


In [ ]:
#test
